In [1]:
import numpy as np
import math
import matplotlib.pyplot

## Q1

Dans cette première modélisation simplifiée, on considère donc que le prix $P_t$ est un processus de Poisson de paramètres $\lambda, \nu$ où $\nu$ est la loi des incréments $J_n$. 

Pour un temps d'attente moyen entre deux sauts de $300s$, on prend $\lambda = \dfrac{1}{300}$


In [2]:
# On identifie le processus par le processus de Poisson composé, qui finit à un temps fixé T
# On crée des fonctions pour modéliser le processus.

#### On définit nu, la loi des incréments
# Ancien : plus lent
saut_1_ancien = lambda x: np.random.choice([-1, 1], size=x, replace=True, p=[0.5, 0.5]) #correspond à m=1
saut_2_ancien = lambda x: np.random.choice([-3, -2, -1, 1, 2, 3], size=x, replace=True, p=0.5*np.array([1/6, 1/3, 1/2, 1/2, 1/3, 1/6])) #correspond à m=3

# Nouveau : beaucoup plus vite
value_1 = np.array([-1, 1])
value_2 = np.array([-3, -2, -2, -1, -1, -1, 1, 1, 1, 2, 2, 3])
saut_1 = lambda x : value_1[np.random.randint(low=2, size=x)]
saut_2 = lambda x : value_2[np.random.randint(low=12, size=x)]

# Les paramètres
P0 = 35
T = 4*60*60
lamb = 1/300

###  Q1 - 2 Quantile


Pour les intervalles de confiance des quantiles, voir cours 2 p.3 (y a un TCL pour le quantile)

__Aussi, parle des approximations gaussiennes des poissons composés p6__

D'après le cours:

Si la loi des sauts est centrée et de variance $\sigma^2$
Si on note $N_T$ le nombre de sauts dans $[0,T]$,

$$N \sim \mathcal{P}(\lambda T)$$
Alors le prix à $T$, $X_T$,  converge en loi vers $\mathcal{N}(P_0,\lambda T \sigma^2)$ quand $\lambda T \rightarrow + \infty$

Ici on a $\lambda T = 48$

Ca serait intéressant de comparer les quantiles obtenus en Q1.2 avec les quantiles de la loi normale $\mathcal{N}(P_0,\lambda T \sigma^2)$, où $\sigma^2=1$ si $m=1$ et $\sigma^2 = ...$ si $m=3$ (normalement, devraient être proches)

In [3]:
#Monte-Carlo accéléré - Méthode 2

# Poisson composé naïf
def Poisson_compose(P0, T, lamb, saut, size):
  # saut : une fonction qui prend la taille de sortie comme entrée pour les sauts.
  N = np.random.poisson(lam=lamb * T, size=size + 1)
  N[0] = 0
  N = np.cumsum(N)                  # La valeur N[i] - N[i - 1] est égale à N pour le i-ième échantillon
                                    # Donc la somme des sauts entre indice N[i] + 1 et N[i + 1] suit la loi voulue
  J = saut(N[-1] + 1)               # Sauts
  J = np.cumsum(J)
  M = J[N[1:]] - J[N[:-1]]          # On veut la somme entre l'indice N[i] + 1 et N[i + 1]
  return P0 + M # échantillon de taille (size)

def quantile(echantillion, size, quant):
  index_bot = int(math.ceil(size * quant))
  index_top = int(math.ceil(size * (1 - quant)))
  echantillion.partition((index_bot, index_top))
  return echantillion[index_bot], echantillion[index_top]

def Question1_2_naive(size, quant, saut, P0, T, lamb):
  return quantile(Poisson_compose(P0, T, lamb, saut, size), size, quant)

### m = 1

In [4]:
%%time
size = int(1e7)
quant = 1e-4
print(Question1_2_naive(size, quant, saut_1, P0, T, lamb))

#Donne 9 et 61 pour P0 = 35 et m=1 et niveau = 10^-4 
#Donne 5 et 65 pour P0 = 35 et m=1 et niveau = 10^-5 
#Donne 3 et 68 pour P0 = 35 et m=1 et niveau = 10^-6
#Donne -13 et 83 pour P0 = 35 et m=2 et niveau = 10^-4 
#Donne -21 et 92 pour P0 = 35 et m=2 et niveau = 10^-5
#Donne -25 et 94 pour P0 = 35 et m=2 et niveau = 10^-6

(9, 61)
CPU times: user 4.28 s, sys: 1.6 s, total: 5.88 s
Wall time: 5.89 s


In [5]:
%%time
quant = 1e-5
print(Question1_2_naive(size, quant, saut_1, P0, T, lamb))

(5, 65)
CPU times: user 4.33 s, sys: 1.62 s, total: 5.94 s
Wall time: 5.95 s


In [6]:
%%time
quant = 1e-6
print(Question1_2_naive(size, quant, saut_1, P0, T, lamb))

(1, 69)
CPU times: user 4.2 s, sys: 1.66 s, total: 5.86 s
Wall time: 5.87 s


### m = 3

In [7]:
%%time
quant = 1e-4
print(Question1_2_naive(size, quant, saut_2, P0, T, lamb))

(-13, 83)
CPU times: user 6.25 s, sys: 1.59 s, total: 7.85 s
Wall time: 7.85 s


In [8]:
%%time
quant = 1e-5
print(Question1_2_naive(size, quant, saut_2, P0, T, lamb))

(-20, 90)
CPU times: user 6.29 s, sys: 1.7 s, total: 7.99 s
Wall time: 8 s


In [9]:
%%time
quant = 1e-6
print(Question1_2_naive(size, quant, saut_2, P0, T, lamb))

(-27, 96)
CPU times: user 6.28 s, sys: 1.6 s, total: 7.88 s
Wall time: 7.9 s


## MCMC

In [40]:
#MCMC (chaînes de Markov) - Méthode 3

def Quantile_ergodique(n,lamb,T,p,P0,saut): #renvoie le quantile de niveau "seuil" du processus de poisson composé

    liste_prix_final = np.zeros(n)

    liste_sauts = liste_sts(lamb,T,saut)

    for l in range(n):
        coloriage = liste_sauts[:,np.random.binomial(1,p,size = len(liste_sauts[0])) ==1]
        liste_sauts_tilde = liste_sts((1-p)*lamb,T,saut)
        nc = coloriage.shape[1]
        nt = liste_sauts_tilde.shape[1]
        liste_sauts = np.zeros((2, nc + nt))
        new_liste_sts(coloriage, liste_sauts_tilde, liste_sauts, nc, nt)

        #new_liste_sauts = np.concatenate((coloriage,liste_sauts_tilde),axis=1)
        #liste_sauts = tri_temps(new_liste_sauts)

        liste_prix_final[l] = np.cumsum(liste_sauts[1,:])[-1] +P0

    liste_prix_final.sort()
    return liste_prix_final

def liste_sts(lbda, T, saut):
    N = np.random.poisson(lbda*T)
    liste_sauts = np.zeros((2, N))
    liste_sauts[0, :] = np.sort(np.random.uniform(low=0, high=T, size=N))
    liste_sauts[1, :] = saut(N)
    #renvoie un array de N colonnes et 2 lignes: 1ere ligne pour les temps des sauts (T_n), deuxième ligne pour leurs amplitudes (J_n)
    return liste_sauts
    #liste_temps_sauts = np.random.uniform(low = 0, high = T, size = N)
    #liste_temps_sauts_triee = [np.sort(liste_temps_sauts)]
    #liste_increments = [saut(N)]
    #return np.concatenate((liste_temps_sauts_triee,liste_increments),axis=0)

from numba import jit
@jit(nopython=True)
def new_liste_sts(coloriage, liste_sauts_tilde, new_liste_sauts, nc, nt):
    i = 0
    j = 0
    k = 0
    # Merge two listes triées
    while i < nc:
        T1 = coloriage[0, i]
        while j < nt and liste_sauts_tilde[0, j] < T1:
            new_liste_sauts[:, k] = liste_sauts_tilde[:, j]
            j += 1
            k += 1
        new_liste_sauts[:, k] = coloriage[:, i]
        i += 1
        k += 1
    while j < nt:
        new_liste_sauts[:, k] = liste_sauts_tilde[:, j]
        j += 1
        k += 1
    return new_liste_sauts

In [42]:
%%time
P0 = 35
T = 4*60*60 #conversion en secondes
lamb =  1/300 
n = int(1e7) 

#niveau = 1e-4
p=0.5

# m = 3
quantiles = Quantile_ergodique(n, lamb,T, p, P0,saut_2) #mettre Quantile_ergodique(niveau,n,lamb,T,p,P0,saut) ou Quantile_naif(niveau,n,lamb,T,P0,saut)
#quantileDroit = Quantile_naif(1-niveau,n,lamb,T,P0,saut_2)
#print("Quantile de niveau {} : {}".format(niveau,quantileGauche))
#print("Quantile de niveau {} : {}".format(1-niveau,quantileDroit))

#Résultats pour Quantile_ergodique:
#Donne 9 et 61 pour P0 = 35 et k=0 et niveau = 10^-4 (c'est à peu près sym/P0 ce qui est rassurant)
#Donne 5 et 66 pour P0 = 35 et k=0 et niveau = 10^-5
#Donne 5 et 63 pour P0 = 35 et k=0 et niveau = 10^-6 (problème... faut augmenter n?) n : 10^5 -> 10^6 : on trouve 2 et 68 ; OK :D

#-15 et 83 pour P0=35 et k=1 et niveau = 10^-4 (sauts plus grands dc on va plus loin, logique; sym/0, logique)
#-16 et 90 pour P0=35 et k=1 et niveau = 10^-5
#Donne -25 et 97 pour P0 = 35 et k=1 et niveau = 10^-6 (en augmentant n de 10^5 à 10^6)

#Résultats Quantile_naïf (plus lent) (n=10^5):
#9 et 62
#5 et 66
#3 et 67
#-13 et 84
#-22 et 87 si on garde n=10^5, -20 et 89 si on passe à n=10^6
# -22 et 97(on passe à n=10^6)

CPU times: user 7min 54s, sys: 112 ms, total: 7min 54s
Wall time: 7min 54s


In [49]:
print(quantiles[9], quantiles[99], quantiles[999], quantiles[9999000], quantiles[9999900], quantiles[9999990])

-27.0 -20.0 -13.0 83.0 90.0 96.0


In [50]:
%%time
# m = 1
quantiles = Quantile_ergodique(n, lamb,T, p, P0,saut_1)
print(quantiles[9], quantiles[99], quantiles[999], quantiles[9999000], quantiles[9999900], quantiles[9999990])

2.0 5.0 9.0 61.0 65.0 68.0
CPU times: user 7min 52s, sys: 51.7 ms, total: 7min 52s
Wall time: 7min 53s


## Echantillionnage d'importance

In [6]:
## Echantillionage d'importance
def quantiles_importance(P0, T, lamb, m, size, f):
  if m == 1:
    value = np.array([-1, 1])
    p = np.array([1/2, 1/2])
  else:
    value = np.array([-3, -2, -1, 1, 2, 3])
    p = np.array([1 / 12, 1 / 6, 1 / 4, 1 / 4, 1 / 6, 1 / 12])
    
  # Nouvelle loi
  s = np.sum(np.exp(f[value]) * p)
  new_lamb = lamb * s
  new_p = np.exp(f[value]) * p / s

  N = np.random.poisson(lam=new_lamb * T, size=size + 1)
  N[0] = 0
  N = N.cumsum()                    # La valeur N[i] - N[i - 1] est égale à Ni pour le i-ième échantillon
                                    # Donc la somme des sauts entre indice N[i] et N[i + 1] - 1 suit la loi voulue
  values_L_T = np.zeros((size, 2))

  J = np.random.choice(value, size=N[-1] + 1, p=new_p)
  X_T_f = f[J]

  J = np.cumsum(J)
  X_T_f = np.cumsum(X_T_f)

  values_L_T[:, 0] = P0 + J[N[1:]] - J[N[:-1]]          # On veut la somme entre l'indice N[i] + 1 et N[i + 1]
  values_L_T[:, 1] = 1. / (np.exp(X_T_f[N[1:]] - X_T_f[N[:-1]] - (s - 1) * lamb * T)) / size

  values_L_T = values_L_T[(-values_L_T[:, 0]).argsort()]             # sort by P_T
  values_L_T[:, 1] = values_L_T[:, 1].cumsum()
  return values_L_T # échantillon de taille (size)

In [7]:
%%time
c = 0.7
values = np.array([0, 1, -1])
f = c * values
values_L_T = quantiles_importance(P0, T, lamb, 1, int(1e7), f)

quantiles = [1e-4, 1e-5, 1e-6]
for quantile in quantiles:
    print(values_L_T[values_L_T[:, 1] < quantile][-1][0])

CPU times: user 15.3 s, sys: 4.85 s, total: 20.1 s
Wall time: 20.2 s


In [51]:
%%time
c = 0.3
values = np.array([0, 1, 2, 3, -1, -2, -3])
f = c * values
values_L_T = quantiles_importance(P0, T, lamb, 3, int(1e7), f)
quantiles = [1e-4, 1e-5, 1e-6]
for quantile in quantiles:
    print(values_L_T[values_L_T[:, 1] < quantile][-1][0])

83.0
90.0
97.0
CPU times: user 17.1 s, sys: 4.35 s, total: 21.5 s
Wall time: 21.5 s


In [50]:
c = 0.25
values = np.array([0, 1, 2, 3, -1, -2, -3])
f = c * values
values_L_T = quantiles_importance(P0, T, lamb, 3, int(1e3), f)
values_L_T[np.where(values_L_T[:, 0] == 70)[0], -1][-1]

0.003072906320918247

In [0]:
#Calcul quantile par échantillonnage d'importance
#Voir slides amphi 2 p.12 pour calcul quantile avec changement de probabilité
#Ensuite voir amphi 3 pour changements de proba Poisson composés
#Différence importance sampling/échantillonage d'importance et changement de probabilité ?
#Transfo d'Esscher